In [1]:
import pandas as pd
import torch 
from pytorch_pretrained_bert import BertTokenizer
from metal.mmtl.metal_model import MetalModel
from metal.mmtl.BERT_tasks import create_tasks
import metal.mmtl.dataset as dataset
from metal.mmtl.scorer import Scorer

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [2]:
#debug_path = '/dfs/scratch0/jdunnmon/mmtl/sota_quest/debugging'
task_name = 'QQP' # SST2
#model_path = f'{debug_path}/{task_name}/best_model.pth'
model_path = '/dfs/scratch0/jdunnmon/mmtl/sota_quest/debugging/QQP/QQP_16_59_01/best_model.pth'
bert_model = 'bert-base-uncased'
max_len = 256
bert_output_dim = 768
dl_kwargs = {"batch_size": 32, 'shuffle': False}

In [3]:
dataset_cls = getattr(dataset, task_name.upper() + "Dataset")
dev_ds = dataset_cls(
    split="dev",
    bert_model=bert_model,
    max_len=max_len,
    max_datapoints=-1,
)
dev_dl = dev_ds.get_dataloader(**dl_kwargs)

HBox(children=(IntProgress(value=0, max=40431), HTML(value='')))

In [5]:
tasks = create_tasks(
        task_names=[task_name],
        bert_model=bert_model,
        split_prop=0.8,
        max_len=max_len,
        dl_kwargs={"batch_size": 100},
        bert_output_dim=bert_output_dim,
        max_datapoints=10,
    )

Loading QQP Dataset


HBox(children=(IntProgress(value=0, max=10), HTML(value='')))

HBox(children=(IntProgress(value=0, max=10), HTML(value='')))

In [21]:
model = MetalModel(tasks, verbose=False, device=0)
model.load_state_dict(torch.load(model_path, map_location="cuda:0")['model'])

In [35]:
from torch.functional import F 

In [69]:
data['score']

array([46.793507, 43.380085, 48.445694, 46.844936, 31.693005, 43.994755,
       49.732437, 41.525158, 47.353104, 43.832718, 35.974487, 36.520267,
       38.844215, 32.555107, 40.40361 , 47.689762, 44.1818  , 54.408707,
       41.89775 , 39.29661 , 31.620132, 44.707558, 42.49611 , 46.38057 ,
       44.788654, 45.51502 , 49.395363, 50.144524, 37.687656, 50.163044,
       45.36488 , 28.923447], dtype=float32)

In [74]:
data = {
    'sentence1': [],
    'sentence2': [],
    'label': [],
    'score' : []
}
tokenizer = BertTokenizer.from_pretrained(bert_model, do_lower_case=True)
from tqdm import tqdm
count = 0
for x, y in tqdm(list(dev_dl)):
    for tokens_idx in x[0]:
        tokens = tokenizer.convert_ids_to_tokens(tokens_idx.numpy())
        phrases = ' '.join(tokens).replace('[PAD]', '').replace('[CLS]', '').split('[SEP]')
        data['sentence1'] += [phrases[0]]
        if len(phrases) > 1:
            data['sentence2'] += [phrases[1]] 
        else:
            data['sentence2'] += ['NA']
    scores = model.calculate_output(x, [task_name])[task_name].detach().cpu().numpy()[:, 1]
    data['score'] += list(scores)
    data['label'] += list(y.numpy()-1)
    count += 1
    if count > 10:
        break

  1%|          | 10/1264 [00:00<01:50, 11.35it/s]


In [68]:
list(scores) + list(scores)

[0.003697698,
 0.5451855,
 0.007516004,
 0.006174406,
 0.003522727,
 0.00200034,
 0.5923954,
 0.044819828,
 0.004346899,
 0.7924309,
 0.011828337,
 0.002286419,
 0.91028255,
 0.002245455,
 0.8296232,
 0.0024239407,
 0.0033225522,
 0.0049559157,
 0.29728317,
 0.0030415428,
 0.50048834,
 0.017027862,
 0.3651014,
 0.92293644,
 0.6411615,
 0.0041479887,
 0.16469052,
 0.04517349,
 0.2719512,
 0.0042476295,
 0.0025178636,
 0.005106384,
 0.003697698,
 0.5451855,
 0.007516004,
 0.006174406,
 0.003522727,
 0.00200034,
 0.5923954,
 0.044819828,
 0.004346899,
 0.7924309,
 0.011828337,
 0.002286419,
 0.91028255,
 0.002245455,
 0.8296232,
 0.0024239407,
 0.0033225522,
 0.0049559157,
 0.29728317,
 0.0030415428,
 0.50048834,
 0.017027862,
 0.3651014,
 0.92293644,
 0.6411615,
 0.0041479887,
 0.16469052,
 0.04517349,
 0.2719512,
 0.0042476295,
 0.0025178636,
 0.005106384]

In [65]:
scores

array([0.00250607, 0.14718229, 0.42612544, 0.00408027, 0.00357515,
       0.00278206, 0.43637222, 0.06134542, 0.00847597, 0.7905831 ,
       0.01813562, 0.00424237, 0.8512961 , 0.0046817 , 0.7468262 ,
       0.0045977 , 0.06299097, 0.00832599, 0.1878354 , 0.00256175,
       0.64360446, 0.00701395, 0.5995938 , 0.91158754, 0.82627434,
       0.00291665, 0.21825962, 0.02787062, 0.00290764, 0.00335525,
       0.00278677, 0.00558513], dtype=float32)

In [64]:
[(x, len(data[x])) for x in data]

[('sentence1', 85016), ('sentence2', 85016), ('label', 85116), ('score', 32)]

In [53]:
df_error = pd.DataFrame(data, columns=['sentence1', 'sentence2', 'score', 'label'])

ValueError: arrays must all be same length

In [25]:
df_error.head(10)

,sentence1,sentence2,score,label
0,why are african - americans so beautiful ?,why are hispanic ##s so beautiful ?,0,1
1,i want to pursue phd in computer science abou...,i handle social media for a non - profit . sh...,0,1
2,is there a reason why we should travel alone ?,what are some reasons to travel alone ?,0,2
3,why are people so obsessed with having a girl...,how can a single male have a child ?,0,1
4,what are some good baby girl names starting w...,what are some good baby girl names starting w...,0,1
5,i am 25 year old guy and never had a girlfrie...,i am 25 years old . i have never had a girlfr...,0,2
6,""" what does a good answer on quo ##ra look li...",how do you write a good answer on quo ##ra ?,0,2
7,what are the best things to do in hong kong ?,what is the best thing in hong kong ?,0,2
8,why is my life getting so complicated ?,why is my life so complicated ?,0,1
9,why should i crack je ##e ?,is it possible to get into ii ##ms with low m...,0,1


In [87]:
df_error.to_csv(f'{debug_path}/{task_name}/dev_error_analysis.tsv', sep='\t')

In [26]:
df_error.shape

(80860, 4)